In [1]:
import re
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from pathlib import Path
import urllib.request
import pandas as pd
import numpy as np

In [2]:
url = "https://www.imdb.com/title/tt0435761"

In [3]:
columnsName = ['movieId', 'genres', 'imdbId', 'imdb_title_id', 'title',
       'original_title', 'year', 'date_published', 'genre', 'duration',
       'country', 'language', 'director', 'writer', 'production_company',
       'actors', 'description', 'avg_vote', 'votes', 'budget',
       'usa_gross_income', 'worlwide_gross_income', 'metascore',
       'reviews_from_users', 'reviews_from_critics']

df = pd.DataFrame(columns=columnsName)

In [4]:
# r = requests.get(url, allow_redirects=True)
# html_doc = r.text

In [5]:
def initialize_webdriver(url):
    options = webdriver.ChromeOptions()
    # options.add_argument("--enable-javascript")
    options.headless = True

    driver = webdriver.Chrome('./chromedriver.exe', options=options) # geckodriver.exe
    driver.implicitly_wait(10)
    driver.get(url) 

    # this is just to ensure that the page is loaded
    time.sleep(12) 
    
    
#     #### For Scroll infinite in page    
#     SCROLL_PAUSE_TIME = 0.5
#     # Get scroll height
#     last_height = driver.execute_script("return document.body.scrollHeight")
#     while True:
#         # Scroll down to bottom
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         # Wait to load page
#         time.sleep(SCROLL_PAUSE_TIME)
#         # Calculate new scroll height and compare with last scroll height
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:
#             break
#         last_height = new_height
        
    html_doc = driver.page_source

    driver.close()

    return html_doc

In [6]:
html_doc = initialize_webdriver(url)

In [7]:
# # https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser

soup = BeautifulSoup(html_doc, 'html5lib')  # 'html5lib' , 'html.parser'  'lxml'

# print(soup.prettify())

In [8]:
string = soup.find('div', class_='title_wrapper').h1.text
pattern = "(.*)\xa0\((\d+)\)"

match = re.search( pattern , string) 

df.loc[0,'title'] = match.group(1)
df.loc[0,'year'] = match.group(2)

In [9]:
string = soup.find('div', class_='ratingValue').strong.get('title')
pattern = "(\d*\.?\d*) based on (\d*\,?\d*)"

match = re.search( pattern , string) 

df.loc[0,'avg_vote'] = match.group(1)  # ratingValue
df.loc[0,'votes'] = match.group(2)  # ratingCount

In [10]:
df.loc[0,'duration'] = soup.find('div', class_='title_wrapper').time.text.strip()

In [11]:
# df['genre'] = '|'.join( genre.text.strip() for genre in soup.find_all('div', class_='see-more inline canwrap')[1].find_all('a') )
df.loc[0,'genre'] = '|'.join( genre.text.strip() for genre in soup.find('div', class_='title_wrapper').find_all('a')[1:-1] )

In [12]:
df.loc[0,'date_published'] = soup.find('div', class_='title_wrapper').find_all('a')[-1].text.strip()  # Release_Date

In [13]:
df.loc[0,'metascore'] = soup.find('div', class_='metacriticScore score_favorable titleReviewBarSubItem').text.strip()

In [14]:
df.loc[0,'description'] = soup.find('div', class_='inline canwrap').text.strip()  # Storyline
df.loc[0,'description'] = soup.find_all('div', class_='summary_text')[0].text.strip() # summary

In [15]:
df.loc[0,'reviews_from_users'] = soup.find('div', class_='titleReviewBarItem titleReviewbarItemBorder').find_all('a')[0].text # '926 user'
df.loc[0,'reviews_from_critics'] = soup.find('div', class_='titleReviewBarItem titleReviewbarItemBorder').find_all('a')[1].text  # '481 critic'

In [16]:
df.loc[0,'director'] = soup.find_all('div', class_='credit_summary_item')[0].a.text

In [17]:
# df['writer'] = 
soup.find_all('div', class_='credit_summary_item')[1].find_all('a')

[<a href="/name/nm0005124/?ref_=tt_ov_wr">John Lasseter</a>,
 <a href="/name/nm0004056/?ref_=tt_ov_wr">Andrew Stanton</a>,
 <a href="fullcredits?ref_=tt_ov_wr#writers/">2 more credits</a>]

In [18]:
# df['actors'] = 
soup.find_all('div', class_='credit_summary_item')[2].find_all('a')

[<a href="/name/nm0000158/?ref_=tt_ov_st_sm">Tom Hanks</a>,
 <a href="/name/nm0000741/?ref_=tt_ov_st_sm">Tim Allen</a>,
 <a href="/name/nm0000349/?ref_=tt_ov_st_sm">Joan Cusack</a>,
 <a href="fullcredits/?ref_=tt_ov_st_sm">See full cast &amp; crew</a>]

In [19]:
# for item in soup.find_all('div', class_='txt-block') :
#     try : 
#         h4text = item.h4.text
#         print(h4text)   
#         try : 
#             val = item.a.text
#             print("\t1)", val)
#         except Exception as e:
#             val = item.text
#             print("\t2)", val)
        
#     except Exception as e:
#         print(f"\tERROR: {e}")

In [20]:
for item in soup.find_all('div', class_='txt-block') :
    try : 
        h4text = item.h4.text
        
        if "Country" in h4text :
            df.loc[0,'country'] = item.a.text

        if "Language" in h4text :
            df.loc[0,'language'] = item.a.text
            
        if "Release Date" in h4text :
            df.loc[0,'date_published'] = re.search( 'Release Date: (.*)\n' , item.text).group(1)

        if "Budget" in h4text :
            df.loc[0,'budget'] = re.search( 'Budget:(.*)\n' , item.text).group(1)   # '$200,000,000'
            
        if "Gross USA:" in h4text :
            df.loc[0,'usa_gross_income'] = re.search( 'Gross USA: (.*)' , item.text.strip() ).group(1)  # '$415,004,880'

        if "Cumulative Worldwide Gross:" in h4text :
            df.loc[0,'worlwide_gross_income'] = re.search( 'Cumulative Worldwide Gross: (.*)' , item.text.strip() ).group(1)  # '$1,066,970,811'
            
        if "Runtime:" in h4text :
            df.loc[0,'duration'] = re.search( 'Runtime:\n(\d*)' , item.text.strip() ).group(1)   # '103'
            
        if "Production Co:" in h4text :
            break
            df.loc[0,'production_company'] = re.search( 'Production Co:\n (.*) ' , item.text.strip() ).group(1)
        
        
    except Exception as e:
        print(f"\tERROR: {e}")

In [21]:
html_doc = initialize_webdriver(url + '/releaseinfo')
soup = BeautifulSoup(html_doc, 'html5lib')  # 'html5lib' , 'html.parser'  'lxml'

In [22]:
for item in soup.find_all('tr', class_='ipl-zebra-list__item aka-item') :
    if "original title" in item.td.text :
        df.loc[0,'original_title'] = item.find_all("td")[1].text 

In [24]:
df.loc[0]

movieId                                                                NaN
genres                                                                 NaN
imdbId                                                                 NaN
imdb_title_id                                                          NaN
title                                                          Toy Story 3
original_title                                                 Toy Story 3
year                                                                  2010
date_published                                          18 June 2010 (USA)
genre                                           Animation|Adventure|Comedy
duration                                                          1h 43min
country                                                                USA
language                                                           English
director                                                       Lee Unkrich
writer                   